In [7]:
import boto3
import json
import pandas as pd
import time


In [68]:
preprocessing = [
    {
        "update_col": "price", 
        "from_col": "price",
        "fn": lambda x: int(x.split(" ")[0].replace("$", "").replace(",", "")),
    },
    {
        "update_col": "score", 
        "from_col": "score",
        "fn": lambda x: float(x),
    },
    {
        "update_col": "num_tickets", 
        "from_col": "num_tickets",
        "fn": lambda x: x.split(" ")[2],
    },
    {
        "update_col": "section", 
        "from_col": "section",
        "fn": lambda x: x.replace("Section ", "").replace("Row ", "") + ", NA",
    },
    {
        "update_col": "row", 
        "from_col": "section",
        "fn": lambda x: x.split(", ")[1],
    },
    {
        "update_col": "section", 
        "from_col": "section",
        "fn": lambda x: x.split(", ")[0],
    },
]

In [5]:
bucket = "seatgeek-tickets"
table = "2023-01-16-09-26-43-941432.json"
s3 = boto3.resource('s3')

obj = s3.Object(bucket, table)
data = json.loads(obj.get()['Body'].read())

In [69]:
dfs = []
for k, v in data.items():
    df = pd.DataFrame(v)
    df['game'] = k
    dfs.append(df)
dfs = pd.concat(dfs)

for p in preprocessing:
    print(p)
    update_col = p['update_col']
    from_col = p['from_col']
    function = p['fn']
    dfs[update_col] = dfs[from_col].apply(function)

{'update_col': 'price', 'from_col': 'price', 'fn': <function <lambda> at 0x000001EF288F39D0>}
{'update_col': 'score', 'from_col': 'score', 'fn': <function <lambda> at 0x000001EF288F3700>}
{'update_col': 'num_tickets', 'from_col': 'num_tickets', 'fn': <function <lambda> at 0x000001EF289AF820>}
{'update_col': 'section', 'from_col': 'section', 'fn': <function <lambda> at 0x000001EF289AF940>}
{'update_col': 'row', 'from_col': 'section', 'fn': <function <lambda> at 0x000001EF289AF8B0>}
{'update_col': 'section', 'from_col': 'section', 'fn': <function <lambda> at 0x000001EF289AF9D0>}
